In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

In [2]:
for file in os.listdir("data"):
    print(file, "\t", os.path.getsize(f"data/{file}") / 1e6, "MB")

sample_submission.csv 	 6.080314 MB
test_identity.csv 	 25.797161 MB
test_transaction.csv 	 613.194934 MB
train_identity.csv 	 26.52968 MB
train_transaction.csv 	 683.351067 MB


In [3]:
df_train_identity = pd.read_csv("data/train_identity.csv")
df_train_transaction = pd.read_csv("data/train_transaction.csv")

In [ ]:
def generateSignature(dataframe, attributes):
    identity_attributes = df_train_transaction[attributes].dropna()
    unique_accounts = identity_attributes.drop_duplicates()
    
    numeric_part = identity_attributes[["card1", "card2", "card3", "card5"]].astype(int)
    numeric_part = numeric_part.astype(str)
    numeric_part = numeric_part[["card1", "card2", "card3", "card5"]].agg("".join, axis=1)
    
    first_char = identity_attributes["card4"].str[0]
    
    second_char = identity_attributes["card6"].apply(len).astype(str)
    
    result = numeric_part + first_char + second_char
    
    return result

# ProductCD Attribute - the payment service attribute

In [ ]:
df_train_transaction.ProductCD.value_counts()

# Filtering Out Information : Selecting Attributes

In [ ]:
meaningful_attributes = [
    "TransactionID",
    "isFraud",
    "TransactionDT",
    "TransactionAmt",
    "ProductCD",
    "card1",
    "card2",
    "card3",
    "card4",
    "card5",
    "card6",
    "addr1",
    "addr2",
    "dist1",
    "dist2",
    "P_emaildomain",
    "R_emaildomain",
]

In [ ]:
df_train_transaction = df_train_transaction[meaningful_attributes]

In [ ]:
df_train_transaction.shape

In [ ]:
df_train_identity = df_train_identity[["TransactionID", "DeviceType", "DeviceInfo"]]

In [ ]:
df_train_identity.shape

In [ ]:
full_dataset = pd.merge(df_train_identity, df_train_transaction, on="TransactionID")

In [ ]:
full_dataset.to_csv("fraud_detection_dataset.csv")

# Map Device Info/Transaction Amount to Unique Accounts

In [ ]:
def createBuckets(num_bins, attribute, df):
    # First, we need to calculate our IQR.
    q1 = df[attribute].quantile(0.25)                 
    q3 = df[attribute].quantile(0.75)
    iqr = q3 - q1

    # Now let's calculate upper and lower bounds.
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr

    # Let us create our bins:
    num_bins = 10
    bin_width = (upper - lower) / num_bins
    bins = [lower + i*bin_width for i in range(num_bins)]
    bins += [upper, float('inf')]  # Now we add our last bin, which will contain any value greater than the upper-bound of the IQR.

    # Let us create our labels:
    labels = [f"Range{i}" for i in range(1,num_bins+1)]
    labels.append("Outlier")

    # Finally, we add a new column to the df:
    df["PriceCategory"] = pd.cut(df[attribute], bins=bins, labels=labels)
    
    return df